In [1]:
import pandas as pd
from auxiliary.filter_functions import filter_chandra, filter_archival, filter_gaia, filter_ned, filter_simbad, filter_erosita, filter_vizier, filter_Xray_binaries
from auxiliary.search_algorithm import get_wcs_event, off_axis, get_chandra_eef, get_counts_from_event
from results import get_candidate_numbers, get_criteria_table
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad, SimbadClass
import os
import glob
from astropy.table import Table
import numpy as np
from astropy.io import fits
import time
import matplotlib.pyplot as plt
from astropy.stats import poisson_conf_interval
import scipy
# 2372400

In [6]:
get_criteria_table(from_date='2015-01-01', to_date='2022-04-01')

,Matched,Unique Matched,Remaining
Archival X-ray date,305,79,161
Cross-match with stars/Gaia,235,38,79
NED + SIMBAD + VizieR,218,12,57


In [5]:
get_candidate_numbers(from_date='2015-01-01', to_date='2022-04-01')

,Total,Before,In,After
Observations,9799,8303,3311,4807
Detections,2802,2701,466,567
Candidates no match,429,412,57,74


In [4]:
detections = pd.read_csv('output/detections_w20.txt',
                         header=0, sep=' ', dtype=str)

detections = detections[detections['ObsId'] == '8490'][0:1]
for index, detection in detections.iterrows():
    print(index, filter_Xray_binaries(detection, True))

  ID           Name          RAJ2000   DEJ2000    Flux    b_Flux   B_Flux  Flagp Flagu Flagn    c       LX      b_LX     B_LX  
                               deg       deg    mW / m2  mW / m2  mW / m2                            1e-07 W  1e-07 W  1e-07 W 
----- --------------------- --------- --------- -------- -------- -------- ----- ----- ----- ------- -------- -------- --------
46957 2CXO J132458.3-430226 201.24330 -43.04060        0        0 9.78e-17     0     1     0 0.50004        0        0 1.45e+35
0 True
